In [ ]:
# from google.colab import runtime
# runtime.unassign()


In [ ]:
!nvidia-smi -L  # Покажет тип GPU

GPU 0: Tesla T4 (UUID: GPU-d9331b1a-64f7-0328-f537-3fc87b7315b0)


In [ ]:
!pip install ultralytics

In [ ]:
#!pip install albumentations ultralytics

In [ ]:
from ultralytics import YOLO
import os
from google.colab import drive
from google.colab.patches import cv2_imshow
import cv2

In [ ]:
DATASET_PATH = "/content/drive/MyDrive/Aircraft_detections/YOLO_detected"

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Проверка структуры датасета
print("Содержимое папки с датасетом:")
print(os.listdir(DATASET_PATH))

# Путь к data.yaml
DATA_YAML_PATH = os.path.join(DATASET_PATH, 'data.yaml')
print(f"\nПуть к data.yaml: {DATA_YAML_PATH}")

Содержимое папки с датасетом:
['test', 'train', 'val', 'data.yaml']

Путь к data.yaml: /content/drive/MyDrive/Aircraft_detections/YOLO_detected/data.yaml


In [ ]:
# Конфигурация для YOLOv11
config = {
    # БЛОК 1: ОСНОВНЫЕ НАСТРОЙКИ ОБУЧЕНИЯ
    'data': DATA_YAML_PATH,           # Путь к YAML-файлу с описанием структуры датасета
    'epochs': 150,                    # Общее количество эпох обучения
    'batch': 8,                      # Размер батча
    'imgsz': 640,                    # Размер входного изображения
    'device': 'cuda',                 # Использование GPU для вычислений

    # БЛОК 2: НАСТРОЙКИ ДАННЫХ И ПРЕПРОЦЕССИНГ
    'workers': 6,                     # Число процессов загрузки данных
    'cache': 'ram',                   # Кэширование в RAM
    'single_cls': True,               # Обучение для одного класса

    # БЛОК 3: ОПТИМИЗАЦИЯ И РЕГУЛЯРИЗАЦИЯ
    'optimizer': 'AdamW',             # Алгоритм оптимизации
    'cos_lr': True,                   # Косинусный schedule для learning rate
    'lr0': 0.0005,
    'lrf': 0.01,                      # Конечный LR = lr0 * lrf
    'amp': True,                      # Смешанная точность
    'patience': 10,                   # Более частая ранняя остановка для предотвращения перетренировки

    # БЛОК 4: АУГМЕНТАЦИЯ (ориентировано на точность)
    'augment': True,
    'mosaic': 0.5,                    # Вероятность мозаики
    'hsv_h': 0.05,
    'hsv_s': 0.5,
    'hsv_v': 0.5,
    'close_mosaic': 50,               # Отключение мозаики после 50 эпох

    'flipud': 0.2,                    # Вертикальное отражение
    'fliplr': 0.4,                    # Горизонтальное отражение
    'perspective': 0.0005,            # Перспективные искажения
    'shear': 0.1,                     # Наклон (макс. 10%)
    'mixup': 0.0,                     # Mixup отключен для улучшения точности
    'copy_paste': 0.05,               # Копирование объектов

    # БЛОК 5: СОХРАНЕНИЕ И МОНИТОРИНГ
    'project': '/content/drive/MyDrive/Aircraft_detections/models',
    'name': 'aircraft_yolov11',
    'exist_ok': True,
    'save_period': 5,
    'plots': True,


    # 'resume': True                  # Раскомментировать для продолжения обучения
}

# Загрузка модели YOLOv11
model = YOLO('yolo11m.pt')
# model = YOLO('/content/drive/MyDrive/Aircraft_detections/saved_models/airport/airport_yolov11/weights/last.pt')
results = model.train(**config)

Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=50, cls=0.5, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/Aircraft_detections/YOLO_detected/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=True, fliplr=0.4, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.05, hsv_s=0.5, hsv_v=0.5, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=0.5, multi_scale=False, name=aircraft_yolov11, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=10, perspec

train: Scanning /content/drive/MyDrive/Aircraft_detections/YOLO_detected/train/labels.cache... 72 images, 0 backgrounds, 36 corrupt: 100%|██████████| 72/72 [00:00<?, ?it/s]

train: /content/drive/MyDrive/Aircraft_detections/YOLO_detected/train/images/014de911-7810-4f7d-8967-3e5402209f4a_jpg.rf.ddeaa26e8a26a0604261f7d7cbf28c15.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: /content/drive/MyDrive/Aircraft_detections/YOLO_detected/train/images/0263270b-e3ee-41dc-aeef-43ff77e66d5b_jpg.rf.2588cf4e0c5533c7631acc67768f3224.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: /content/drive/MyDrive/Aircraft_detections/YOLO_detected/train/images/22291e0b-ebe2-4f3f-b53e-4e709179300a_jpg.rf.b6f17caa296b3eb820821503e939b818.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: /content/drive/MyDrive/Aircraft_detections/YOLO_detected/train/images/2314c1b5-ec8f-4212-b42f-43365a13fd20_jpg.rf.e0f8efb25c6f53ab0858fbb48c0b0529.jpg: ignoring corrupt image/label: Label class 1 excee


train: Caching images (0.0GB RAM): 100%|██████████| 36/36 [00:00<00:00, 213.97it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 1.4±1.3 ms, read: 34.9±4.0 MB/s, size: 80.7 KB)


...

     38/150      5.35G      1.218     0.5614     0.9208        165        640: 100%|██████████| 5/5 [00:01<00:00,  3.43it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.65it/s]

                   all         10        244      0.974      0.917      0.954       0.62


EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 28, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

38 epochs completed in 0.036 hours.
Optimizer stripped from /content/drive/MyDrive/Aircraft_detections/models/aircraft_yolov11/weights/last.pt, 40.5MB
Optimizer stripped from /content/drive/MyDrive/Aircraft_detections/models/aircraft_yolov11/weights/best.pt, 40.5MB

Validating /content/drive/MyDrive/Aircraft_detections/models/aircraft_yolov11/weights/best.pt...
Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.97it/s]


                   all         10        244      0.983      0.898      0.955      0.646
Speed: 0.2ms preprocess, 40.8ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to /content/drive/MyDrive/Aircraft_detections/models/aircraft_yolov11
